# Cleaning the main data (BankChurners.csv)

In [208]:
import pandas as pd

In [210]:
df = pd.read_csv("../data/BankChurners.csv")

### Drop irrelevant columns

In [213]:
df = df.drop([
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'
], axis=1)
df = df.drop(columns=['Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Contacts_Count_12_mon',
                                  'Total_Ct_Chng_Q4_Q1'])

## Analysing the dataframe

In [216]:
display(df.head(5))

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,12691.0,777,1144,42,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,8256.0,864,1291,33,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,3418.0,0,1887,20,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,3313.0,2517,1171,20,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,4716.0,0,816,28,0.000


### Renaming columns & naming for Income Category

In [219]:
df = df.rename(columns={'Months_on_book' : 'Month_with_bank',
                        'Total_Relationship_Count' : 'No_of_product',
                        'Total_Trans_Ct' : 'Total_Trans_Count'})

#### Income Category

In [222]:
def clean_col(x):
        if 'K' in x:
            return x.replace('K','').replace('$','')
        elif '+' in x:
            return x.replace('+','')
        elif x =='Less than 40':
            return x.split()[2]
        return x
    
df['Income_Category'] = df['Income_Category'].apply(clean_col)

In [224]:
shape = df.shape
shape

(10127, 17)

### Duplicates

In [227]:
duplicates = df.duplicated().sum()
duplicates

0

### NAs

In [230]:
na_values = df.isna().sum()
na_values

CLIENTNUM                 0
Attrition_Flag            0
Customer_Age              0
Gender                    0
Dependent_count           0
Education_Level           0
Marital_Status            0
Income_Category           0
Card_Category             0
Month_with_bank           0
No_of_product             0
Months_Inactive_12_mon    0
Credit_Limit              0
Total_Revolving_Bal       0
Total_Trans_Amt           0
Total_Trans_Count         0
Avg_Utilization_Ratio     0
dtype: int64

### Categorical variables

In [233]:
categorical = ['Attrition_Flag', 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']
for col in categorical:
    print(df[col].unique())

for category in categorical:
    df[category] = df[category].astype('category')

['Existing Customer' 'Attrited Customer']
['M' 'F']
['High School' 'Graduate' 'Uneducated' 'Unknown' 'College' 'Post-Graduate'
 'Doctorate']
['Married' 'Single' 'Unknown' 'Divorced']
['60 - 80' 'Less than 40' '80 - 120' '40 - 60' '120 +' 'Unknown']
['Blue' 'Gold' 'Silver' 'Platinum']


### Numerical factors

In [236]:
numerical_columns = [
    'Customer_Age', 'Dependent_count', 'Month_with_bank', 'No_of_product',
    'Months_Inactive_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Total_Trans_Amt', 'Total_Trans_Count',
    'Avg_Utilization_Ratio'
]

df[numerical_columns] = df[numerical_columns].apply(pd.to_numeric, errors='coerce')

df.dtypes

CLIENTNUM                    int64
Attrition_Flag            category
Customer_Age                 int64
Gender                    category
Dependent_count              int64
Education_Level           category
Marital_Status            category
Income_Category           category
Card_Category             category
Month_with_bank              int64
No_of_product                int64
Months_Inactive_12_mon       int64
Credit_Limit               float64
Total_Revolving_Bal          int64
Total_Trans_Amt              int64
Total_Trans_Count            int64
Avg_Utilization_Ratio      float64
dtype: object

### Unknowns

In [239]:
unknown_income_row = df[df['Income_Category'] == 'Unknown'].shape[0]
total_row = df.shape[0]
unknown_income_percentage = (unknown_income_row / total_row) * 100
print(f"Percentage of rows with 'Unknown' income: {unknown_income_percentage:.2f}%")

#Considering that these rows are less than 15% of the whole dataset, we simply drop it instead of filling it up with cross validation method
#This is also to avoid overfitting

unknown_education_row = df[df['Education_Level'] == 'Unknown'].shape[0]
unknown_education_percentage = (unknown_education_row / total_row) * 100
print(f"Percentage of rows with 'Unknown' education: {unknown_education_percentage:.2f}%")

unknown_martial_status_row = df[df['Marital_Status'] == 'Unknown'].shape[0]
unknown_martial_status_percentage = (unknown_martial_status_row / total_row) * 100
print(f"Percentage of rows with 'Unknown' martial status: {unknown_martial_status_percentage:.2f}%")

total_unknow = df[(df['Education_Level'] == 'Unknown') | (df['Marital_Status'] == 'Unknown') | (df['Income_Category'] == 'Unknown')]
total_unknown_percentage = (total_unknow.shape[0] / total_row) * 100
print(f"Percentage of rows with 'Unknown' total: {total_unknown_percentage:.2f}%")


Percentage of rows with 'Unknown' income: 10.98%
Percentage of rows with 'Unknown' education: 15.00%
Percentage of rows with 'Unknown' martial status: 7.40%
Percentage of rows with 'Unknown' total: 30.08%


### Downloading the file

In [242]:
df.to_csv('BankChurners_cleaned.csv', index=False)